In [1]:
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path='../Paper1-RAGG/impl/.env')

### OpenAI
os.environ["TOGETHER_API_KEY"] = os.getenv("TOGETHER_API_KEY")

In [7]:
import replicate
from langchain_community.llms import Replicate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document

[nltk_data] Downloading package punkt to /home/hamzicd/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
### embedding
model2 = "ehsanaghaei/SecureBERT"
embeddings2 = HuggingFaceEmbeddings(model_name = model2)

/tmp/ipykernel_2566231/3801564067.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings2 = HuggingFaceEmbeddings(model_name = model2)
/home/hamzicd/.local/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
No sentence-transformers model found with name ehsanaghaei/SecureBERT. Creating a new one with mean pooling.
Some weights of RobertaModel were not initialized from the model checkpoint at ehsanaghaei/SecureBERT and are ne

In [9]:
db = FAISS.load_local("./TTPFShot-training-data-19747", embeddings2, allow_dangerous_deserialization=True)

In [38]:
def predict_label_llama_prompt_builder(text_to_predict):
    ##### Prompt 1
    search_results = db.similarity_search_with_score(text_to_predict, k=65)
    
    #print(search_results)

    
    xs = ["\nText:"+ doc.page_content.lower().strip()+"\nSimilarity:"+ str(score) +"\nClassification:"+doc.metadata['TTP_ID']+"\n" for doc, score in search_results if score < 20]
    #xs = ["\nText:'"+ doc.page_content.lower().strip()+"'\nSimilarity:"+ str(score) +"\nClassification:"+doc.metadata['TTP_ID']+"\n" for doc, score in search_results]
    

    xs = ''.join(xs)+"\nInputText:"+ text_to_predict.lower().strip() +"\nClassification:"
    xs = """You are a MITRE ATT&CK TTP classification expert.
    Your task is to classify the 'InputText' based on the provided examples below.
    Each example shows:
    - a sentence (Text),
    - inverse similarity to InputText (the lower the better)(Similarity),
    - and its corresponding classification label starting with 'T'.
    
    Use these examples to determine the correct classification for the given text (InputText).
    
    If 'InputText' is completely not related to any of the provided examples, return class 'T0000'.
    To determine if the 'InputText' is not related to the given examples, you can use the 'Similarity' propery of the examples.
    Low 'Similarity' values indicate the high similarity in strings.
    Return only the classification label starting with 'T' or 'T0000' if you found no appropriate class/label for 'InputText'.\n
    Do not output any outher string/text or explanation except the label starting with 'T' or 'T0000' if you found no label(class).
    Ouput only the label from the provided samples, no other texts and no explanations!
    Output only the label!
    """ + xs
    
    #####
    #**Think through the classification process step-by-step, comparing the 'InputText' to each example, but do not share your reasoning.**
    
    #print(xs)
    prompt = xs
    #result = llm.invoke(prompt)
    #predicted_label = result.content
    #print(prompt)
    return prompt#predicted_label

In [70]:
df_reports = pd.read_csv("./df_reports_classification_CISA-77-llama-3-70b.csv", index_col=False)
df_reports

,Unnamed: 0.1,Unnamed: 0,RawText,TTP,CleanText,ttpfshot_sl_os_llama-3-70b
0,0,https://www.cisa.gov/news-events/cybersecurity...,\n\n\n\n\n\n\nActions to take today to mitigat...,"{'T1083', 'T1071.002', 'T1055.004', 'T1657', '...",\n\n\n\n\n\n\nActions to take today to mitigat...,"['T1021.001', 'T1210', 'T1497', 'T1597.001', '..."
1,1,https://www.cisa.gov/news-events/cybersecurity...,\n\n\n\nHow SVR-Attributed Actors are Adapting...,"{'T1621', 'T1090.002', 'T1110', 'T1078.004', '...",\n\n\n\nHow SVR-Attributed Actors are Adapting...,"['T1021.001', 'T1210', 'T1562.001', 'T1480.001..."
2,2,https://www.cisa.gov/news-events/cybersecurity...,\n\n\n\n\n\n\nActions to take today to mitigat...,"{'T1021.007', 'T1083', 'T1087.002', 'T1552.001...",\n\n\n\n\n\n\nActions to take today to mitigat...,"['T1021.001', 'T1210', 'T1497', 'T1597.001', '..."
3,3,https://www.cisa.gov/news-events/cybersecurity...,\n\n\n\n\n\n\nActions to take today to mitigat...,"{'TA0003', 'TA0008', 'T1083', 'T1654', 'T1078....",\n\n\n\n\n\n\nActions to take today to mitigat...,"['T1021.001', 'T1210', 'T1497', 'T1480.001', '..."
4,4,https://www.cisa.gov/news-events/cybersecurity...,\n\n\n\n\n\n\nActions to take today to mitigat...,"{'TA0006', 'T1595.002', 'T1190', 'T1083', 'T15...",\n\n\n\n\n\n\nActions to take today to mitigat...,"['T1133', 'T1210', 'T1497', 'T1597.001', 'T148..."
...,...,...,...,...,...,...
72,72,https://www.cisa.gov/news-events/cybersecurity...,\n\n\n\n\nSummary\n\nThis joint cybersecurity ...,"{'TA0003', 'T1190', 'TA0008', 'T1110', 'T1189'...",\n\n\n\n\nSummary\n\nThis joint cybersecurity ...,"['T1021.001', 'T1210', 'T1562.001', 'T1597.001..."
73,73,https://www.cisa.gov/news-events/cybersecurity...,\n\n\n\n\nSummary\n\nThis Alert uses the MITRE...,"{'T1041', 'T1573.002', 'T1566.002', 'T1021.002...",\n\n\n\n\nSummary\n\nThis Alert uses the MITRE...,"['T1021.001', 'T1210', 'T1497', 'T1480.001', '..."
74,74,https://www.cisa.gov/news-events/cybersecurity...,\n\n\n\n\nSummary\n\nThis Alert uses the MITRE...,"{'T1041', 'T1055.012', 'T1027.002', 'T1546.008...",\n\n\n\n\nSummary\n\nThis Alert uses the MITRE...,"['T1021.001', 'T1210', 'T1562.001', 'T1480.001..."
75,75,https://www.cisa.gov/news-events/cybersecurity...,\n\n\n\n\nSummary\n\nThis Alert uses the MITRE...,"{'T1083', 'T1016', 'T1036.005', 'T1105', 'T108...",\n\n\n\n\nSummary\n\nThis Alert uses the MITRE...,"['T1021.001', 'T1210', 'T1562.001', 'T1480.001..."


In [72]:
from together import Together
import pprint
import io

client = Together()


def call_llm(model, prompt):
    #print("calling llm...")
    stream = client.chat.completions.create(
      model=model,
      messages=[{"role": "user", "content": prompt}],
      stream=False,
    )
    
    return str(stream.choices[0].message.content)
    #print(stream)

In [65]:
MODEL = "mistralai/Mixtral-8x22B-Instruct-v0.1"
prompt = "How are you?"

call_llm(MODEL, prompt)

calling llm...


" I'm an artificial intelligence and don't have feelings, but I'm here and ready to assist you. How can I help you today?"

In [28]:
#df_reports = df_reports.head(2)

## Run Benchmark

In [73]:
from together import Together
import time

import io
import re

client = Together()


#MODEL = "mistralai/Mixtral-8x22B-Instruct-v0.1"
MODEL = "DzenanH/mistralai/Mixtral-8x7B-Instruct-v0.1-46a293ec"

#df_reports["ttpfshot_sl_os_llama-3-70b"] = ""
df_reports["ttpfshot_sl_os_mixtral"] = ""

for index, row in df_reports.iterrows():
    #print(f"Index: {index}")
    print("text nr: ", index)
    
    text_to_predict = row["CleanText"]
    
    start_time = time.time()

    sentences = sent_tokenize(text_to_predict)

    predicted_labels = []
    for sentence in sentences:#[0:8]:

        prompt = predict_label_llama_prompt_builder(sentence)
        
        output_value = call_llm(MODEL, prompt)
        #print(output_value)
        
        # Regular expression to match T followed by digits, optionally followed by a dot and more digits
        match = re.search(r'T\d{4}(\.\d{3})?', output_value)
        
        # Extract the matched value
        if match:
            #print("REGEX")
            output_value = match.group(0)  # Extract the matched value
            print(output_value)
        
        # Clean up the output (if needed)
        predicted_label = output_value.replace('\n', '')  # Remove newlines if any
        predicted_labels.append(predicted_label)
        
        # Print the predicted label
        print("sentence predicted:", predicted_label)


    print("--------")
    # Remove 'T0000' if it exists
    if 'T0000' in predicted_labels:
        predicted_labels.remove('T0000')
    
    print(predicted_labels)
    
    elapsed_time = (time.time() - start_time)/60
    print(f"ttpfshot_sl_OS done in: {elapsed_time} minutes")

    ### save results to the reports dataframe
    #df_reports.at[index, "ttpfshot_sl_os_llama-3-70b"] = predicted_labels
    df_reports.at[index, "ttpfshot_sl_os_mixtral"] = predicted_labels

text nr:  0
T1076
sentence predicted: T1076
T0000
sentence predicted: T0000
T1562
sentence predicted: T1562
T0000
sentence predicted: T0000
T0000
sentence predicted: T0000
T0000
sentence predicted: T0000
T1497
sentence predicted: T1497
T1486
sentence predicted: T1486
T1486
sentence predicted: T1486
T1496
sentence predicted: T1496
T0000
sentence predicted: T0000
T0000
sentence predicted: T0000
T0000
sentence predicted: T0000
T0000
sentence predicted: T0000
T1588.001
sentence predicted: T1588.001
T1110.003
sentence predicted: T1110.003
T1589.001
sentence predicted: T1589.001
T1219
sentence predicted: T1219
T1105
sentence predicted: T1105
T1059.003
sentence predicted: T1059.003
T1059.003
sentence predicted: T1059.003
T1055.001
sentence predicted: T1055.001
T1090.002
sentence predicted: T1090.002
T1140
sentence predicted: T1140
T1562.004
sentence predicted: T1562.004
T1564.002
sentence predicted: T1564.002
T1543.003
sentence predicted: T1543.003
T1547.001
sentence predicted: T1547.001
T107

In [33]:
df_reports

,Unnamed: 0.1,Unnamed: 0,RawText,TTP,CleanText,ttpfshot_sl_os_llama-3-70b,ttpfshot_sl_os_mixtral
0,0,https://www.cisa.gov/news-events/cybersecurity...,\n\n\n\n\n\n\nActions to take today to mitigat...,"{'T1083', 'T1071.002', 'T1055.004', 'T1657', '...",\n\n\n\n\n\n\nActions to take today to mitigat...,"['T1021.001', 'T1210', 'T1497', 'T1597.001', '...","[ T1076, T1190, T1562.004, T1486, T1588.006]"
1,1,https://www.cisa.gov/news-events/cybersecurity...,\n\n\n\nHow SVR-Attributed Actors are Adapting...,"{'T1621', 'T1090.002', 'T1110', 'T1078.004', '...",\n\n\n\nHow SVR-Attributed Actors are Adapting...,"['T1021.001', 'T1210', 'T1562.001', 'T1480.001...","[ T1583.001, T1588.001, T0000, T1133, T0000]"


In [74]:
df_reports.to_csv("./df_reports_classification_CISA-77-llama-3-70b-mixtral.csv")

In [58]:
# Function to clean the 'ttpfshot_sl_os_mixtral' column
def clean_ttp_column(ttps):
    # Remove 'T0000' and ensure unique elements
    return list(sorted(set(ttps) - {'T0000'}))

# Apply the cleaning function to the column
df_reports['ttpfshot_sl_os_mixtral'] = df_reports['ttpfshot_sl_os_mixtral'].apply(clean_ttp_column)
df_reports['ttpfshot_sl_os_llama-3-70b'] = df_reports['ttpfshot_sl_os_llama-3-70b'].apply(clean_ttp_column)

T1486
